# 正規表現

## 概要

In [176]:
import re

* https://docs.python.org/3/library/re.html
    * http://docs.python.jp/3/library/re.html （日本語）
* `re` をインポートすると**正規表現 (regular expression)**によるあいまいな文字列の検索が可能になる

### `search` メソッド

In [218]:
prog = re.compile('my favou?rite colou?r is gr[ae]y')

match = prog.search('As you know, my favorite color is gray.')
print(match)
match = prog.search('As you know, my favourite colour is grey.')
print(match)
match = prog.search('As you know, my favorite color is orange.')
print(match)

<_sre.SRE_Match object; span=(13, 38), match='my favorite color is gray'>
<_sre.SRE_Match object; span=(13, 40), match='my favourite colour is grey'>
None


* re.compile 関数で正規表現オブジェクトを作成し、`search` メソッドを呼ぶことで検索できる
* マッチする文字列が存在した場合は `match` オブジェクトが返り、存在しない場合は `None` が返る

### `match` オブジェクト

In [223]:
s = 'As you know, my favourite colour is grey.'
match = prog.search(s)
print(match)
print(match.group())
print(match.span(), match.start(), match.end())

<_sre.SRE_Match object; span=(13, 40), match='my favourite colour is grey'>
my favourite colour is grey
(13, 40) 13 40


* マッチした文字列は `match` オブジェクトの `group` メソッドで得られる
* マッチした範囲は `span` メソッドで得られる
    * あるいは `start`, `end` メソッドで始点と終点を得られる
    * 上の例では、13 文字目から **39** 文字目までにマッチしている（40 文字目は含まれない）
        * `s[13:40] == match.group()` である

### `match` メソッド

In [227]:
match = prog.match('As you know, my favorite color is gray.')
print(match)
match = prog.match('my favorite color is gray.')
print(match)

None
<_sre.SRE_Match object; span=(0, 25), match='my favorite color is gray'>


* `search` メソッドの代わりに `match` メソッドを使った場合、文字列の先頭からマッチするかどうかを判定できる
    * 先頭に余分な文字列が入っている場合マッチに失敗する

### `fullmatch` メソッド

In [230]:
match = prog.fullmatch('As you know, my favorite color is gray.')
print(match)
match = prog.fullmatch('my favorite color is gray.')
print(match)
match = prog.fullmatch('my favorite color is gray')
print(match)

None
None
<_sre.SRE_Match object; span=(0, 25), match='my favorite color is gray'>


* `fullmatch` メソッドは文字列がパターンに完全に一致した際にのみ `match` オブジェクトを返す
    * 先頭にも末尾にも余分な文字列が入っていてはならない

### `sub` メソッド

In [250]:
prog = re.compile('\d')
print(prog.sub('*', '012-3456-7890'))

***-****-****


* `sub` メソッドで文字列を置換することができる

### 捕捉グループ

In [259]:
prog = re.compile('^([^:]+): (\S+)$')
s = 'Top: ../index.html'
print(prog.match(s).group(0))
print(prog.match(s).group(1))
print(prog.match(s).group(2))
print(prog.sub(r'<a href='\2'>\1</a>', s))

Top: ../index.html
Top
../index.html
<a href="../index.html">Top</a>


* 正規表現に括弧を含めた場合、括弧内の表現にマッチする部分文字列が**捕捉 (capture)**される
    * 捕捉した文字列は `group(n)` で参照できる
    * `group(0)` はマッチした文字列全体を表す（引数を省略すると 0 として扱われる）
* `sub` での置換文字列中では `\1`, `\2`, ... のようにして参照できる

## 量指定子

In [140]:
def check_if_fullmatch(prog, strs):
    match_dict = {True: [], False: []}
    for s in strs:
        match = prog.search(s)
        match_dict[bool(match)].append(s)
    if match_dict[True]:
        print(prog.pattern, '[ OK ]', ', '.join(match_dict[True]))
    if match_dict[False]:
        print(prog.pattern, '[ NG ]', ', '.join(match_dict[False]))

### 0 個以上にマッチ

In [141]:
strs = ['bt', 'bot', 'boot', 'boat', 'booooooot']

prog = re.compile('bo*t')
check_if_fullmatch(prog, strs)

bo*t [ OK ] bt, bot, boot, booooooot
bo*t [ NG ] boat


### 1 個以上にマッチ

In [142]:
prog = re.compile('bo+t')
check_if_fullmatch(prog, strs)

bo+t [ OK ] bot, boot, booooooot
bo+t [ NG ] bt, boat


### 0 個か 1 個ならマッチ

In [143]:
prog = re.compile('bo?t')
check_if_fullmatch(prog, strs)

bo?t [ OK ] bt, bot
bo?t [ NG ] boot, boat, booooooot


### 指定の個数にマッチ

In [144]:
prog = re.compile('bo{1,2}t')
check_if_fullmatch(prog, strs)

bo{1,2}t [ OK ] bot, boot
bo{1,2}t [ NG ] bt, boat, booooooot


## 括弧によるグループ化

In [145]:
prog = re.compile('(ass)*in')
check_if_fullmatch(prog, ['in', 'assassin'])

(ass)*in [ OK ] in, assassin


## OR 検索

In [148]:
prog = re.compile('dog|cat')
check_if_fullmatch(prog, ['dog', 'cat', 'cow'])

dog|cat [ OK ] dog, cat
dog|cat [ NG ] cow


In [151]:
prog = re.compile('(in|de)flation')
check_if_fullmatch(prog, ['inflation', 'deflation', 'stagflation'])

(in|de)flation [ OK ] inflation, deflation
(in|de)flation [ NG ] stagflation


## メタキャラクタ

### 任意の文字にマッチ

In [172]:
strs = ['bat', 'bit', 'but', 'bet', 'bot',
        'bAt', 'b1t', 'b t', 'b-t', 'b_t']

prog = re.compile('b.t')
check_if_fullmatch(prog, strs)

b.t [ OK ] bat, bit, but, bet, bot, bAt, b1t, b t, b-t, b_t


### 指定の文字にマッチ

In [174]:
prog = re.compile('b[aoe]t')
check_if_fullmatch(prog, strs)

b[aoe]t [ OK ] bat, bet, bot
b[aoe]t [ NG ] bit, but, bAt, b1t, b t, b-t, b_t


In [175]:
prog = re.compile('b[a-z]t')
check_if_fullmatch(prog, strs)

b[a-z]t [ OK ] bat, bit, but, bet, bot
b[a-z]t [ NG ] bAt, b1t, b t, b-t, b_t


* 範囲指定もできる（文字コード順）

In [176]:
prog = re.compile(r'b[a-z\-]t')
check_if_fullmatch(prog, strs)

b[a-z\-]t [ OK ] bat, bit, but, bet, bot, b-t
b[a-z\-]t [ NG ] bAt, b1t, b t, b_t


* `‐` 自体を含めたい場合にはバックスラッシュを使う

### 指定以外の文字にマッチ

In [181]:
prog = re.compile(r'b[^aoe]t')
check_if_fullmatch(prog, strs)

b[^aoe]t [ OK ] bit, but, bAt, b1t, b t, b-t, b_t
b[^aoe]t [ NG ] bat, bet, bot


### 単語を構成する文字にマッチ

In [177]:
prog = re.compile(r'b\wt')
check_if_fullmatch(prog, strs)

b\wt [ OK ] bat, bit, but, bet, bot, bAt, b1t, b_t
b\wt [ NG ] b t, b-t


### 数値にマッチ

In [178]:
prog = re.compile(r'b\dt')
check_if_fullmatch(prog, strs)

b\dt [ OK ] b1t
b\dt [ NG ] bat, bit, but, bet, bot, bAt, b t, b-t, b_t


### 数値以外にマッチ

In [180]:
prog = re.compile(r'b\Dt')
check_if_fullmatch(prog, strs)

b\Dt [ OK ] bat, bit, but, bet, bot, bAt, b t, b-t, b_t
b\Dt [ NG ] b1t


### 空白文字にマッチ

In [182]:
prog = re.compile(r'b\st')
check_if_fullmatch(prog, strs)

b\st [ OK ] b t
b\st [ NG ] bat, bit, but, bet, bot, bAt, b1t, b-t, b_t


### 空白以外の文字にマッチ

In [183]:
prog = re.compile(r'b\St')
check_if_fullmatch(prog, strs)

b\St [ OK ] bat, bit, but, bet, bot, bAt, b1t, b-t, b_t
b\St [ NG ] b t


## アンカー

In [11]:
def scan(prog, s):
    match_strs = []
    pos = 0
    while pos < len(s):
        match = prog.search(s, pos=pos)
        if match:
            pos = match.end()
            match_strs.append(match.group())
        else:
            break
    return match_strs

### 行頭

In [304]:
s = '''CO2
H2O
Na2CO3
H2CO3
CH3COOH
CH4
CO
O2'''

prog = re.compile(r'^H.*', re.MULTILINE)
print(scan(prog, s))

['H2O', 'H2CO3']


### 行末

In [303]:
prog = re.compile(r'.*O$', re.MULTILINE)
print(scan(prog, s))

['H2O', 'CO']


### 単語境界

In [324]:
s = 'CO2 H2O Na2CO3 H2CO3 CH3COOH CH4 CO O2'
prog = re.compile(r'C\S+')
print(scan(prog, s))
prog = re.compile(r'\bC\S+')
print(scan(prog, s))

['CO2', 'CO3', 'CO3', 'CH3COOH', 'CH4', 'CO']
['CO2', 'CH3COOH', 'CH4', 'CO']


## 貪欲マッチ／最小一致マッチ

In [15]:
s = 'regular expression'
prog = re.compile(r'r.+r')
print(scan(prog, s))
prog = re.compile(r'r.+?r')
print(scan(prog, s))

['regular expr']
['regular']


* 正規表現はデフォルトで貪欲 (greedy) マッチを行う
    * できるだけ長い文字列にマッチするような挙動になっている
* 量指定子 (`*`, `+`, `?`, `{m,n}`) の後に `?` を付けることで非貪欲 (non-greedy) ／最小一致 (minimal) マッチにできる

# Example

## Atomic Weight

* [files/atoms.txt](files/atoms.txt) は http://www.chem.qmul.ac.uk/iupac/AtWt/ の Table 2 をブラウザからそのままコピー＆ペーストしたファイルである
* このファイルを元素記号をキー、原子量を値とする辞書にうまく変換したい
* 原子量に `( )` が入っていたり `[ ]` が付いていたりして数値に変換できない

In [3]:
import re

# 「float らしき文字列」を表す正規表現
float_re = re.compile(r'\d+(\.\d+)?')

line = '2	He	Helium	4.002602(2)	1, 2'
row = line.split('\t')
print(row)
print(float_re.search(row[3]).group())

line = '43	Tc	Technetium	[97]	4'
row = line.split('\t')
print(row)
print(float_re.search(row[3]).group())

['2', 'He', 'Helium', '4.002602(2)', '1, 2']
4.002602
['43', 'Tc', 'Technetium', '[97]', '4']
97


* 「`float` らしき文字列」を検索することにより原子量を抽出できた

In [4]:
atom_dict = {}

with open('files/atoms.txt') as file:
    for i, line in enumerate(file):
        if i == 0:
            # 最初の行 (header) は無視する
            continue
        row = line.split('\t')
        # 「float らしき文字列」を抽出する
        weight_str = float_re.search(row[3]).group()
        atom_dict[row[1]] = float(weight_str)

print(atom_dict)

{'Al': 26.9815385, 'Np': 237.0, 'Pu': 244.0, 'Mt': 276.0, 'Te': 127.6, 'Fm': 257.0, 'Ar': 39.948, 'Be': 9.0121831, 'Cl': 35.45, 'Mg': 24.305, 'I': 126.90447, 'Uup': 289.0, 'Db': 270.0, 'Yb': 173.045, 'Sm': 150.36, 'O': 15.999, 'Rn': 222.0, 'Pa': 231.03588, 'Ac': 227.0, 'Dy': 162.5, 'Er': 167.259, 'Re': 186.207, 'Cn': 285.0, 'Ca': 40.078, 'Pb': 207.2, 'Co': 58.933194, 'Ne': 20.1797, 'Xe': 131.293, 'Ge': 72.63, 'Cu': 63.546, 'U': 238.02891, 'Au': 196.966569, 'K': 39.0983, 'Ra': 226.0, 'Ba': 137.327, 'Po': 209.0, 'Gd': 157.25, 'Zn': 65.38, 'Cd': 112.414, 'Tl': 204.38, 'Tm': 168.93422, 'Nd': 144.242, 'Si': 28.085, 'No': 259.0, 'F': 18.998403163, 'Cm': 247.0, 'Ti': 47.867, 'Zr': 91.224, 'Sb': 121.76, 'Es': 252.0, 'S': 32.06, 'Bi': 208.9804, 'Li': 6.94, 'Cf': 251.0, 'He': 4.002602, 'Ho': 164.93033, 'Pd': 106.42, 'Pt': 195.084, 'Eu': 151.964, 'Tc': 97.0, 'Th': 232.0377, 'Lr': 262.0, 'N': 14.007, 'Ds': 281.0, 'V': 50.9415, 'As': 74.921595, 'Cr': 51.9961, 'Ni': 58.6934, 'Lu': 174.9668, 'Sn': 11

## Formula Weight Calculator

* `'Na2CO3'` のような分子式を `Na2`, `C`, `O3` に分けたい

In [5]:
atom_re = re.compile('([A-Z][a-z]{0,2})(\d+)?')

formula = 'Na2CO3'
match = atom_re.match(formula)

print(match.group(1))
print(match.group(2))
print(match.span(), match.start(), match.end())

Na
2
(0, 3) 0 3


* 捕捉グループを使って原子と数を分けることができた

In [6]:
match = atom_re.match(formula, pos=3)

print(match.group(1))
print(match.group(2))
print(match.span(), match.start(), match.end())

C
None
(3, 4) 3 4


* `match` メソッドの `pos=` 引数で検索を開始する位置を指定できる
* すなわち、前回のマッチの `match.end()` を `pos=` に指定することによって次のマッチを検索できる

In [7]:
def create_formula_dict(formula):
    formula_dict = {}
    pos = 0
    while pos < len(formula):
        match = atom_re.match(formula, pos=pos)
        if match:
            pos = match.end()
            atom = match.group(1)
            n_str = match.group(2)
            n = int(n_str) if n_str else 1
            if atom in formula_dict:
                formula_dict[atom] += n
            else:
                formula_dict[atom] = n
        else:
            raise Exception('invalid formula')
    return formula_dict

print(create_formula_dict('H2O'))
print(create_formula_dict('Na2CO3'))
print(create_formula_dict('CH3COOH'))
print(create_formula_dict('C18H36O2'))
print(create_formula_dict('C18H36O2'))

{'O': 1, 'H': 2}
{'Na': 2, 'O': 3, 'C': 1}
{'C': 2, 'O': 2, 'H': 4}
{'C': 18, 'O': 2, 'H': 36}
{'C': 18, 'O': 2, 'H': 36}


* マッチに失敗した場合 `raise` 文で例外 (exception) を投げてエラーを発生させている
* `a if cond else b` は `cond` が `True` なら `a`、`False` なら `b` になる

In [9]:
def calc_formula_weight(formula):
    formula_dict = create_formula_dict(formula)
    weight = 0
    for (sym, n) in formula_dict.items():
        weight += atom_dict[sym] * n
    return weight

print(calc_formula_weight('H2O'))
print(calc_formula_weight('Na2CO3'))
print(calc_formula_weight('CH3COOH'))
print(calc_formula_weight('C18H36O2'))

18.015
105.98753855999999
60.05199999999999
284.484


* 前の例で作った原子量の辞書と組み合わせることによって分子量（厳密には式量）の計算ができる
* 括弧を含む式（`'(COOH)2'` など）には対応していない
    * 括弧のような入れ子（再帰）構造を正規表現で表すことは一般にできない
    * 括弧は別の方法で取り扱う必要がある